In [3]:
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
import os
from PyPDF2 import PdfReader

def extract_text_from_pdfs(directory):
    texts = []
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            filepath = os.path.join(directory, filename)
            reader = PdfReader(filepath)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            texts.append(text)
    return texts

document_texts = extract_text_from_pdfs('/home/lillian/customer-service-agent/data')


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents(document_texts)


In [21]:
from langchain.embeddings import GooglePalmEmbeddings

embedding_model = GooglePalmEmbeddings()


In [26]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [28]:
import chromadb


class EmbeddingFunction:
    def __call__(self, input):
        
        return embeddings.embed_documents(input)


chroma_client = chromadb.Client()


embedding_function = EmbeddingFunction()

collection = chroma_client.create_collection(
    name='company_documents_test_2',
    embedding_function=embedding_function
)


In [25]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [29]:
texts = [doc.page_content for doc in documents]

print(texts)

['GOVERNMENT OF TAMILNADU \nDIRECTORATE OF TECHNICAL EDUCATION \nCHENNAI – 600  025  \n \nSTATE PROJECT COORDINATION UNIT \n \nDiploma in Electronics and Communication \nEngineering  \nCourse Code: 1040  \nN – Scheme  \ne-TEXTBOOK  \non \nCOMPUTER HARDWARE SERVICING AND \nNETWORKING  \nfor \nVI Semester DECE  \n \nConvener for ECE Discipline:  \n \nDr.  M.JEGANMOHAN,M.E.,MBA.,Ph.D.,(Management).,Ph.D.,(Eng g).,M.I.S.T.E.,  \nPrincipal,  \nGovernment Polytechnic College, \nUthappanaickanoor,', 'Uthappanaickanoor, \nUsilampatti, Madurai – 625 536.  \n \nTeam Members for Computer Hardware Servicing and Networ king:  \n \n1. Mrs.S.Nirmala,B.E., \nSenior Lecturer /ECE, \nSakthi polytechnic college, \nErode – 638  315.  \n2. Mr.V.Manikandan, \nLecturer /Computer, \nSakthi polytechnic college, \nErode - 638  315.  \n3. Mrs.A.Velvizhi, M.E.,M.I.S.T.E., \nVice Principal / Computer, \nDevangar Polytechnic College, \nPeriyakulam – 625  602.  \n \nValidated By \n \nDr.S.Mohamed Mansoor Roomi,M.E.,

In [30]:
texts = [doc.page_content for doc in documents]

collection.add(
    documents=texts,
    
    ids=[str(i) for i in range(len(texts))]
)

In [51]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [34]:
test_texts = ["This is a test.", "Another test sentence."]
test_embeddings = embeddings.embed_documents(test_texts)
print(test_embeddings)


[[0.0549314022064209, 0.00519124511629343, -0.0616946779191494, -0.034094683825969696, 0.04694109782576561, 0.014355835504829884, -0.0002622316824272275, -0.041081421077251434, 0.008367808535695076, 0.051672082394361496, -0.019291579723358154, 0.01665104180574417, -0.026872696354985237, 0.012590480037033558, 0.01873106323182583, -0.01917976513504982, 0.004034899175167084, 0.009138963185250759, 0.01796652190387249, 0.006133783608675003, -0.004504537209868431, 0.0126001862809062, -0.009057532995939255, 0.0019072860013693571, 0.03477681800723076, 0.006809446960687637, 0.029981326311826706, -0.03550957143306732, -0.032766684889793396, -0.021528955549001694, -0.05300186574459076, 0.009102239273488522, -0.04911990091204643, 0.02336772531270981, 0.007273700088262558, -0.05594783276319504, -0.012324880808591843, 0.0137120820581913, -0.003537900047376752, -0.01488930732011795, 0.024109717458486557, -0.025883430615067482, -0.025366265326738358, 0.0269639790058136, 0.009964467957615852, 0.0235088

In [106]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(
    texts= texts,
    collection_name="company_documents_test_2",
    embedding=embeddings
)

retriever = vectorstore.as_retriever()


In [56]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [40]:
import os
from dotenv import load_dotenv
import streamlit as st
from PyPDF2 import PdfReader
import pandas as pd
from datetime import datetime
import csv

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import google.generativeai as genai

In [57]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-gecko-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [112]:
document_texts = extract_text_from_pdfs('/home/lillian/customer-service-agent/data')
print("Extracted Document Texts:", document_texts)


Extracted Document Texts: [' \nGOVERNMENT OF TAMILNADU \nDIRECTORATE OF TECHNICAL EDUCATION \nCHENNAI – 600  025  \n \nSTATE PROJECT COORDINATION UNIT \n \nDiploma in Electronics and Communication \nEngineering  \nCourse Code: 1040  \nN – Scheme  \ne-TEXTBOOK  \non \nCOMPUTER HARDWARE SERVICING AND \nNETWORKING  \nfor \nVI Semester DECE  \n \nConvener for ECE Discipline:  \n \nDr.  M.JEGANMOHAN,M.E.,MBA.,Ph.D.,(Management).,Ph.D.,(Eng g).,M.I.S.T.E.,  \nPrincipal,  \nGovernment Polytechnic College, \nUthappanaickanoor, \nUsilampatti, Madurai – 625 536.  \n \nTeam Members for Computer Hardware Servicing and Networ king:  \n \n1. Mrs.S.Nirmala,B.E., \nSenior Lecturer /ECE, \nSakthi polytechnic college, \nErode – 638  315.  \n2. Mr.V.Manikandan, \nLecturer /Computer, \nSakthi polytechnic college, \nErode - 638  315.  \n3. Mrs.A.Velvizhi, M.E.,M.I.S.T.E., \nVice Principal / Computer, \nDevangar Polytechnic College, \nPeriyakulam – 625  602.  \n \nValidated By \n \nDr.S.Mohamed Mansoor Room

In [113]:
documents = text_splitter.create_documents(document_texts)
print("Split Documents:", documents)


Split Documents: [Document(page_content='GOVERNMENT OF TAMILNADU \nDIRECTORATE OF TECHNICAL EDUCATION \nCHENNAI – 600  025  \n \nSTATE PROJECT COORDINATION UNIT \n \nDiploma in Electronics and Communication \nEngineering  \nCourse Code: 1040  \nN – Scheme  \ne-TEXTBOOK  \non \nCOMPUTER HARDWARE SERVICING AND \nNETWORKING  \nfor \nVI Semester DECE  \n \nConvener for ECE Discipline:  \n \nDr.  M.JEGANMOHAN,M.E.,MBA.,Ph.D.,(Management).,Ph.D.,(Eng g).,M.I.S.T.E.,  \nPrincipal,  \nGovernment Polytechnic College, \nUthappanaickanoor,'), Document(page_content='Uthappanaickanoor, \nUsilampatti, Madurai – 625 536.  \n \nTeam Members for Computer Hardware Servicing and Networ king:  \n \n1. Mrs.S.Nirmala,B.E., \nSenior Lecturer /ECE, \nSakthi polytechnic college, \nErode – 638  315.  \n2. Mr.V.Manikandan, \nLecturer /Computer, \nSakthi polytechnic college, \nErode - 638  315.  \n3. Mrs.A.Velvizhi, M.E.,M.I.S.T.E., \nVice Principal / Computer, \nDevangar Polytechnic College, \nPeriyakulam – 625 

In [114]:
from langchain.embeddings import GooglePalmEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

def embedding_function(texts):
    return embeddings.embed_documents(texts)


In [115]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(
    texts=[doc.page_content for doc in documents],
    embedding=embeddings,
    collection_name="company_documents_test_2",
    persist_directory="/home/lillian/customer-service-agent/chroma_db"
)


In [67]:
# from langchain.vectorstores import Chroma

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# def embedding_function(texts):
#     return embeddings.embed_documents(texts)

# vectorstore = Chroma(
#     collection_name='company_documents_test_2',
#     embedding_function=embedding_function
# )


In [116]:
retriever = vectorstore.as_retriever()

In [119]:
query = "What is the full form cpu?"
retrieved_docs = retriever.get_relevant_documents(query)
print("Retrieved Documents:", retrieved_docs)


Retrieved Documents: [Document(page_content='laptops \n(i) DDR  266 PC 2100 \n(ii) DDR 333 PC 2700 \n(iii)DDR 400 PC 3200 \n(iv)DDR2 -400 PC 3200 \n(v) DDR2 – 533 PC 4200 \n \nCPU – types \nThe full form of cpu is control processing unit. This is t he heart of the laptop and is \nused to process the data according to the instructions given by the uses. The main cpu \nmanufactures are intel and AMD. The following are the d ifferent types of laptop cpu’s  \n(i) Intel atom, cleron, pentium M , core is series \n(ii) AMD neo.'), Document(page_content='dedicated L1 and L2 cache, but share a common L3 cache. When an instruction is \nreferenced in the L3 cache, it is typically elevated to a higher tier cache. \n Main memory \n \nThe main memory of the computer is also known as RAM , standing for Random \nAccess Memory. It is constructed from integrated circui ts and needs to have electrical power \nin order to maintain its information. When power is lost,  the information is lost too. It can b

In [91]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro-latest")


In [102]:
from langchain.schema import HumanMessage


messages = [HumanMessage(content="Hello, who are you and who do you work for?")]

response = llm(messages)


print(response.content)


I am Gemini, a multimodal AI language model developed by Google. I am designed to assist and inform users to the best of my abilities. I do not work for any specific organization or entity.


In [131]:
from langchain.prompts import PromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro-latest")

def classify_intent(question):
    prompt = f"""
You are an AI assistant that classifies customer questions into one of the following intents:

- **Inventory Inquiry**: Questions about product availability, specifications, pricing, or inventory.
- **General Inquiry**: Questions about company policies, services, or general information.
- **Other**: Any other questions.

Based on the user's question below, classify it into one of the intents and return only the intent label: 'Inventory Inquiry', 'General Inquiry', or 'Other'.

Question:
"{question}"

Intent:
"""
    
    messages = [HumanMessage(content=prompt)]
   
    response = llm(messages)

    intent = response.content.strip()
    return intent

# prompt = PromptTemplate(
#     template=prompt_template,
#     input_variables=["context", "question"]
# )


In [134]:
user_question = "Who are you?"
intent = classify_intent(user_question)
print(f"Identified Intent: {intent}")


Identified Intent: Other


In [107]:
# from langchain.chains import RetrievalQA
# from langchain.prompts import PromptTemplate

# prompt_template = """
# Answer the question based on the provided context. If the answer is not in the context, say "I'm sorry, but I don't have that information."

# Context:
# {context}

# Question:
# {question}

# Answer:
# """

# prompt = PromptTemplate(
#     template=prompt_template,
#     input_variables=["context", "question"]
# )

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",  
#     retriever=retriever,
#     chain_type_kwargs={"prompt": prompt},
#     return_source_documents=True
# )


In [121]:
# import pandas as pd

# # Load the inventory data
# inventory_df = pd.read_csv('/home/lillian/customer-service-agent/ML-Dataset.csv')

# def search_inventory(question):
#     for index, row in inventory_df.iterrows():
#         product_name = str(row['ProductName']).lower()
#         product_description = str(row['ProductDescription']).lower()
#         if product_name in question.lower() or product_description in question.lower():
#             return row
#     return None


In [94]:
import psycopg2

# Set up the database connection
conn = psycopg2.connect(
    dbname='inventory',
    user='postgres',
    password='1234',
    host='localhost',
    port='5432'
)
cursor = conn.cursor()


In [138]:
def search_inventory(question):
    try:
        
        cursor.execute("SELECT DISTINCT \"productname\" FROM inventory_data;")
        products = cursor.fetchall()
        product_names = [p[0] for p in products if p[0] is not None]

        
        for product_name in product_names:
            if product_name.lower() in question.lower():
                
                cursor.execute("""
                    SELECT *
                    FROM inventory_data
                    WHERE LOWER("productname") = %s
                    LIMIT 1;
                """, (product_name.lower(),))
                result = cursor.fetchone()
                if result:
                    columns = [desc[0] for desc in cursor.description]
                    product_info = dict(zip(columns, result))
                    return product_info
        return None
    except Exception as e:
        print(f"Error searching inventory: {e}")
        conn.rollback()
        return None


In [139]:
product_info = search_inventory("Intel Xeon E5-2699 V3 (OEM/Tray)")
print("Product Info:", product_info)


Product Info: {'regionname': 'South America', 'countryname': 'United States of America', 'state': 'Texas', 'city': 'Southlake', 'postalcode': '26192', 'warehouseaddress': '2014 Jabberwocky Rd', 'warehousename': 'Southlake Texas', 'employeename': 'Summer Payne', 'employeeemail': 'summer.payne@example.com', 'employeephone': '5151238181', 'employeehiredate': datetime.date(2016, 6, 7), 'employeejobtitle': 'Public Accountant', 'categoryname': 'CPU', 'productname': 'Intel Xeon E5-2699 V3 (OEM/Tray)', 'productdescription': 'Speed:2.3GHz,Cores:18,TDP:145W', 'productstandardcost': Decimal('2867.51'), 'profit': Decimal('542.95'), 'productlistprice': Decimal('3410.46'), 'customername': 'Flor Stone', 'customeraddress': '2904 S Salina St', 'customercreditlimit': Decimal('5000.00'), 'customeremail': 'flor.stone@raytheon.com', 'customerphone': '13171234104', 'status': 'Shipped', 'orderdate': datetime.date(2016, 11, 17), 'orderitemquantity': 132, 'perunitprice': Decimal('469.99'), 'totalitemquantity':

In [109]:
def log_interaction(user_question, answer):
    try:
        interaction_timestamp = datetime.now()
        cursor.execute("""
            INSERT INTO user_interactions (interaction_timestamp, user_question, answer)
            VALUES (%s, %s, %s);
        """, (interaction_timestamp, user_question, answer))
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"Error logging interaction: {e}")


In [142]:
def answer_question(user_question):
    intent = classify_intent(user_question)
    context = ""

    
    if intent == 'Inventory Inquiry':
        
        product_info = search_inventory(user_question)
        if product_info:
            context += (
                f"Product Name: {product_info.get('productname', 'N/A')}\n"
                f"Category: {product_info.get('categoryname', 'N/A')}\n"
                f"Description: {product_info.get('productdescription', 'N/A')}\n"
                f"Price: ${product_info.get('productlistprice', 'N/A')}\n"
                f"Quantity in Stock: {product_info.get('totalitemquantity', 'N/A')}\n"
                f"Warehouse: {product_info.get('warehousename', 'N/A')} at {product_info.get('warehouseaddress', 'N/A')}\n"
                f"Contact: {product_info.get('employeename', 'N/A')} ({product_info.get('employeeemail', 'N/A')})\n"
            )
        else:
            context += "No matching products found in inventory.\n"
    elif intent == 'General Inquiry':
       
        retrieved_docs = retriever.get_relevant_documents(user_question)
        if retrieved_docs:
            context += "\n".join([doc.page_content for doc in retrieved_docs])
        else:
            context += "No relevant documents found.\n"
    else:
        
        product_info = search_inventory(user_question)
        if product_info:
            context += (
                f"Product Name: {product_info.get('productname', 'N/A')}\n"
                f"Category: {product_info.get('categoryname', 'N/A')}\n"
                f"Description: {product_info.get('productdescription', 'N/A')}\n"
                f"Price: ${product_info.get('productlistprice', 'N/A')}\n"
                f"Quantity in Stock: {product_info.get('totalitemquantity', 'N/A')}\n"
                f"Warehouse: {product_info.get('warehousename', 'N/A')} at {product_info.get('warehouseaddress', 'N/A')}\n"
                f"Contact: {product_info.get('employeename', 'N/A')} ({product_info.get('employeeemail', 'N/A')})\n"
            )
        
        retrieved_docs = retriever.get_relevant_documents(user_question)
        if retrieved_docs:
            context += "\n".join([doc.page_content for doc in retrieved_docs])

    
    if context.strip():
        prompt = f"""
You are an AI assistant for a company that specializes in computer-based products and services. Your role is to assist customers by providing information from the company's documents and inventory database.

Use the following context to answer the customer's question. If you cannot find the information in the context, politely inform the customer that you don't have that information.

Ensure your response is clear, professional, and informative.

Context:
{context}

Question:
{user_question}

Answer:
"""
        # messages = [HumanMessage(content=prompt)]   
        response = llm([HumanMessage(content=prompt)])
        answer = response.content.strip()
    else:
        answer = "I'm sorry, but I don't have that information."

    log_interaction(user_question, answer)

    return answer


In [110]:
# def answer_question(user_question):

#     product_info = search_inventory(user_question)
#     if product_info:

#         answer = (
#             f"We have {product_info.get('productname', 'N/A')} available.\n"
#             f"Category: {product_info.get('CategoryName', 'N/A')}\n"
#             f"Description: {product_info.get('ProductDescription', 'N/A')}\n"
#             f"Price: ${product_info.get('ProductListPrice', 'N/A')}\n"
#             f"Quantity in stock: {product_info.get('TotalItemQuantity', 'N/A')}\n"
#             f"Warehouse: {product_info.get('WarehouseName', 'N/A')} at {product_info.get('WarehouseAddress', 'N/A')}\n"
#             f"Contact: {product_info.get('EmployeeName', 'N/A')} ({product_info.get('EmployeeEmail', 'N/A')})"
#         )
#     else:

#         answer = qa_chain({'query': user_question})['result']

#     log_interaction(user_question, answer)
#     return answer


In [146]:
def test_rag_pipeline():
    
    user_question = "Do you have Intel Xeon E5-2670 V3, what is its description and who is going to ship it to me?"

    try:
        answer = answer_question(user_question)
        print("User Question:")
        print(user_question)
        print("\nAnswer:")
        print(answer)
    except Exception as e:
        print(f"An error occurred during testing: {e}")

test_rag_pipeline()

User Question:
Do you have Intel Xeon E5-2670 V3, what is its description and who is going to ship it to me?

Answer:
Yes, we have the Intel Xeon E5-2670 V3 in stock. It has a speed of 2.3GHz and 12 cores, and its TDP is 120W. The product is currently located in our warehouse in Southlake, Texas at 2014 Jabberwocky Rd. Unfortunately, I do not have information on who will be shipping the product to you. For more information, please contact Dexter Barnes at dexter.barnes@example.com.
